In [1]:
import numpy as np

In [2]:
point_grid_path = "./out/build/grid_points.dat"
index_map_path = "./out/build/index_map.dat"
shadow_map_path = "./out/build/shadow_map.dat"

azimuth_map_path = "./out/build/azimuth_map.dat"
elevation_map_path = "./out/build/elevation_map.dat"

In [3]:
index_map = np.memmap(index_map_path, dtype=np.uint32, mode='r')
azimuth_map = np.memmap(azimuth_map_path, dtype=np.float16, mode='r')
elevation_map = np.memmap(elevation_map_path, dtype=np.float16, mode='r')
point_grid = np.loadtxt(point_grid_path)

In [9]:
# point_grid_num = 331547
# bbox_min = (84501.6, 445805, -3.747)
# resolution = 2.0
voxel_dim = (587,590,50)

In [4]:
point_grid = point_grid[:,0:6]

In [5]:
irradiance = np.zeros(point_grid.shape[0])

In [50]:
def integrate_voxel_info(point_grid, irradiance_vals, voxel_size = 2.0 ):
    voxel_grid = {}
    bbox_min = np.min(point_grid[:, :3], axis=0)
    
    def compute_intensity_for_face(normals, face_normal):
        ratio = np.sum(normals @ face_normal > 0) / len(normals)
        dot_products = normals[normals @ face_normal > 0] @ face_normal
        if len(dot_products) > 0:
            intensity = np.mean(dot_products) * ratio
        else:
            intensity = 0
        return intensity
    
    voxel_id_list = []

    for i in range(point_grid.shape[0]):
        # point = point_grid[i]
        voxel_ids = ((point_grid[i, :3] - bbox_min) / voxel_size).astype(int)
        voxel_idx = voxel_ids[0] + voxel_ids[1] * voxel_dim[0] + voxel_ids[2] * voxel_dim[0] * voxel_dim[1]
        # voxel_idx = voxel_ids[0] + voxel_ids[1] * voxel_dim[0] + voxel_ids[2] * voxel_dim[0] * voxel_dim[1]
        # print(voxel_ids)
        if voxel_idx not in voxel_grid:
            voxel_grid[voxel_idx] = {'normals': [], 'irradiance': 0}
            voxel_id_list.append(voxel_idx)
        voxel_grid[voxel_idx]['normals'].append(point_grid[i, 3:6])
        voxel_grid[voxel_idx]['irradiance'] += irradiance_vals[i]

    for voxel in voxel_grid:
        normals = np.array(voxel_grid[voxel]['normals'])
        voxel_grid[voxel]['up_intensity'] = compute_intensity_for_face(normals, np.array([0, 0, 1]))
        voxel_grid[voxel]['down_intensity'] = compute_intensity_for_face(normals, np.array([0, 0, -1]))
        voxel_grid[voxel]['left_intensity'] = compute_intensity_for_face(normals, np.array([-1, 0, 0]))
        voxel_grid[voxel]['right_intensity'] = compute_intensity_for_face(normals, np.array([1, 0, 0]))
        voxel_grid[voxel]['front_intensity'] = compute_intensity_for_face(normals, np.array([0, 1, 0]))
        voxel_grid[voxel]['back_intensity'] = compute_intensity_for_face(normals, np.array([0, -1, 0]))

        intensity_sum = (voxel_grid[voxel]['up_intensity'] + voxel_grid[voxel]['down_intensity'] +
                         voxel_grid[voxel]['left_intensity'] + voxel_grid[voxel]['right_intensity'] +
                         voxel_grid[voxel]['front_intensity'] + voxel_grid[voxel]['back_intensity'])

        if intensity_sum > 0:
            for key in ['up_intensity', 'down_intensity', 'left_intensity', 'right_intensity', 'front_intensity', 'back_intensity']:
                voxel_grid[voxel][key] /= intensity_sum

    return voxel_grid, voxel_id_list

# def update_grid_point_irradiance(point_grid, voxel_grid, irradiance, index_map, azimuth_map, elevation_map, voxel_size=2.0):
#     num_points = point_grid.shape[0]
#     num_samples = 360 * 90
#     bbox_min = np.min(point_grid[:, :3], axis=0)
#     voxel_dim = np.ceil((np.max(point_grid[:, :3], axis=0) - bbox_min) / voxel_size).astype(int)

#     for i in range(num_points):
#         voxel_ids = ((point_grid[i, :3] - bbox_min) / voxel_size).astype(int)
#         voxel_idx = voxel_ids[0] + voxel_ids[1] * voxel_dim[0] + voxel_ids[2] * voxel_dim[0] * voxel_dim[1]
#         voxel_data = voxel_grid[voxel_idx]

#         directions = np.stack([np.cos(np.radians(elevation_map[i*num_samples + j])) * np.cos(np.radians(azimuth_map[i*num_samples + j])),
#                                np.cos(np.radians(elevation_map[i*num_samples + j])) * np.sin(np.radians(azimuth_map[i*num_samples + j])),
#                                np.sin(np.radians(elevation_map[i*num_samples + j]))] for j in range(num_samples), axis=-1)

#         mask = index_map[i*num_samples:(i+1)*num_samples] != np.iinfo(np.uint32).max

#         directions = directions[:, mask]

#         contributions = np.zeros(num_samples)
#         valid_directions = directions[:, mask]

#         contributions[mask] += np.where(valid_directions[0, :] > 0, valid_directions[0, :] * voxel_data['right_intensity'], -valid_directions[0, :] * voxel_data['left_intensity'])
#         contributions[mask] += np.where(valid_directions[1, :] > 0, valid_directions[1, :] * voxel_data['front_intensity'], -valid_directions[1, :] * voxel_data['back_intensity'])
#         contributions[mask] += np.where(valid_directions[2, :] > 0, valid_directions[2, :] * voxel_data['up_intensity'], -valid_directions[2, :] * voxel_data['down_intensity'])

#         irradiance[i] += np.sum(contributions * voxel_data['irradiance'] * 0.1)

#         if i % 100 == 0:
#             print('Processed {} points'.format(i))
#     return irradiance


def update_grid_point_irradiance(point_grid, voxel_grid, irradiance, index_map, azimuth_map, elevation_map, voxel_size=2.0):
    num_points = point_grid.shape[0]
    num_samples = 360 * 90
    bbox_min = np.min(point_grid[:, :3], axis=0)
    # bbox_min = np.min(point_grid[:, :3], axis=0)
    # voxel_dim = np.ceil((np.max(point_grid[:, :3], axis=0) - bbox_min) / voxel_size).astype(int)

    for i in range(num_points):
        voxel_ids = ((point_grid[i, :3] - bbox_min) / voxel_size).astype(int)
        voxel_idx = voxel_ids[0] + voxel_ids[1] * voxel_dim[0] + voxel_ids[2] * voxel_dim[0] * voxel_dim[1]
        voxel_data = voxel_grid[voxel_idx]

        directions = np.stack([np.cos(np.radians(elevation_map[i*num_samples + j])) * np.cos(np.radians(azimuth_map[i*num_samples + j])),
                               np.cos(np.radians(elevation_map[i*num_samples + j])) * np.sin(np.radians(azimuth_map[i*num_samples + j])),
                               np.sin(np.radians(elevation_map[i*num_samples + j]))] for j in range(num_samples))

        mask = index_map[i*num_samples:(i+1)*num_samples] != np.iinfo(np.uint32).max

        contributions = np.zeros(num_samples)
        directions = directions[mask]

        contributions[mask] += np.where(directions[:, 0] > 0, directions[:, 0] * voxel_data['right_intensity'], -directions[:, 0] * voxel_data['left_intensity'])
        contributions[mask] += np.where(directions[:, 1] > 0, directions[:, 1] * voxel_data['front_intensity'], -directions[:, 1] * voxel_data['back_intensity'])
        contributions[mask] += np.where(directions[:, 2] > 0, directions[:, 2] * voxel_data['up_intensity'], -directions[:, 2] * voxel_data['down_intensity'])

        irradiance[i] += np.sum(contributions * voxel_data['irradiance'] * 0.1)

        if i % 100 == 0:
            print('Processed {} points'.format(i))
    return irradiance



In [44]:
updated_voxel, voxel_ids = integrate_voxel_info(point_grid, irradiance)


In [51]:
updated_irradiance = update_grid_point_irradiance(point_grid, updated_voxel, irradiance, index_map, azimuth_map, elevation_map)

TypeError: arrays to stack must be passed as a "sequence" type such as list or tuple.

In [48]:
updated_voxel = integrate_voxel_info(point_grid, irradiance)

In [55]:
updated_irradiance = update_grid_point_irradiance(point_grid, updated_voxel, irradiance, index_map, azimuth_map, elevation_map)

Processed 0 points
Processed 100 points


KeyboardInterrupt: 

In [19]:
sparse_voxel = {}

point_grid_num = 331547
bbox_min = (84501.6, 445805, -3.747)
resolution = 2.0
voxel_dim = (587,590,50)
index_map


# point_grid_irradiance = np.array((point_grid_num, 1), dtype=np.float32)
point_grid_irradiance = np.zeros((point_grid_num, ), dtype=np.float32)

sparse_voxel_arr = []

xs = []
ys = []
zs = []

for i in range(point_grid_num):
    coords = point_grid[i][0:3]
    normal = point_grid[i][3:6]
    ix = int((coords[0] - bbox_min[0]) / resolution)
    iy = int((coords[1] - bbox_min[1]) / resolution)
    iz = int((coords[2] - bbox_min[2]) / resolution)
    voxel_idx = ix + iy * voxel_dim[0] + iz * voxel_dim[0] * voxel_dim[1]
    if voxel_idx not in sparse_voxel:
        sparse_voxel[voxel_idx] = []
    sparse_voxel[voxel_idx].append(normal)
    sparse_voxel[voxel_idx].append(point_grid_irradiance[i])
    sparse_voxel_arr.append(voxel_idx)

In [36]:
for voxel in sparse_voxel:
    # data = np.array(sparse_voxel[voxel])

    
    normals = np.array([sparse_voxel[voxel][i] for i in range(0, len(sparse_voxel[voxel]), 2)])
    irradiance_values = np.array([sparse_voxel[voxel][i] for i in range(1, len(sparse_voxel[voxel]), 2)])
    num_points = len(normals)

    elevations = np.degrees(np.arcsin(normals[:, 2]))
    azimuths = np.degrees(np.arctan2(normals[:, 1], normals[:, 0]))

    intensity_up_ratio = np.sum(elevations > 45) / num_points
    intensity_down_ratio = np.sum(elevations < -45) / num_points
    intensity_front_ratio = np.sum((elevations > 0) & (elevations <= 45) & (azimuths >= -135) & (azimuths <= -45)) / num_points
    intensity_back_ratio = np.sum((elevations > 0) & (elevations <= 45) & (azimuths >= 45) & (azimuths <= 135)) / num_points
    intensity_left_ratio = np.sum((elevations > 0) & (elevations <= 45) & ((azimuths >= 135) | (azimuths <= -135))) / num_points
    intensity_right_ratio = np.sum((elevations > 0) & (elevations <= 45) & (azimuths >= -45) & (azimuths <= 45)) / num_points

    sparse_voxel[voxel].append([intensity_up_ratio, intensity_down_ratio, intensity_front_ratio, intensity_back_ratio, intensity_left_ratio, intensity_right_ratio])